# Modelo predictivo de tumores de mama
* Luque Venegas, Italo Israel
* Medrano Vargas, José Luis
* Soto Orbezo, Irvin Rigoberto
* Yance Gutierrez, Franco Felix
* Bonifacio Aldair , Cristian

# Importar librerias

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import joblib
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

# Cargar base de datos

In [2]:

data = pd.read_csv('data.csv', sep=',')

# Compresion de los datos

In [3]:
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Observamos que se tienen 31 variables a analizar y 1 de ellas es la varible objetivo: diagnosis

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave_points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

Observamos ademas que la variable id no sera parte del analisis

In [5]:
# Análisis de valores faltantes
print("\nAnálisis de valores faltantes:")
print(data.isnull().sum())


Análisis de valores faltantes:
id                         0
diagnosis                  0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave_points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave_points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave_points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64


No hay valores faltantes con lo cual no sera necesario preprocesar estos

# Conclusiones
* Se trata de una base de datos medica de las caracteristicas fisicas visibles de diversos tumores encontrados
* La variable objetivo es diagnosis la cual nos indica M si el tumor es maligno y B si es benigno
* No hay valores faltantes
* Al ser base de datos medica no se deberan tratar con valores atipicos ni modificar en lo posible la data
* todos los valores caracteristicos son tipo float

# Preprocesamiento de datos

In [7]:
# Dividir el dataset en características (X) y etiquetas (y)
X = data.drop(['diagnosis','id'], axis=1)
y = data['diagnosis']
#X['Sex'] = X['Sex'].astype('category')

In [8]:
y.value_counts()

diagnosis
B    357
M    212
Name: count, dtype: int64

Observamos que hay un desbalance de clases, no es significativo pero podria afectar al analisis

In [9]:
# Codificación de la columna 'income'
y = y.map({'B': '0', 'M': '1'})
y.value_counts()


diagnosis
0    357
1    212
Name: count, dtype: int64

Codificamos la variable diagnosis tal que de ser maligno se dara 1

In [10]:
y= y.astype('object')
y.value_counts()

diagnosis
0    357
1    212
Name: count, dtype: int64

In [11]:
#Verificamos la correcta separacion
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   radius_mean              569 non-null    float64
 1   texture_mean             569 non-null    float64
 2   perimeter_mean           569 non-null    float64
 3   area_mean                569 non-null    float64
 4   smoothness_mean          569 non-null    float64
 5   compactness_mean         569 non-null    float64
 6   concavity_mean           569 non-null    float64
 7   concave_points_mean      569 non-null    float64
 8   symmetry_mean            569 non-null    float64
 9   fractal_dimension_mean   569 non-null    float64
 10  radius_se                569 non-null    float64
 11  texture_se               569 non-null    float64
 12  perimeter_se             569 non-null    float64
 13  area_se                  569 non-null    float64
 14  smoothness_se            5

In [12]:
# Dividir el dataset en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Conclusiones
* Se tienen ahora 2 data frames para el entrenamiento, se dividieron en 80% para el entrenamiento y 20% para validacion
* La variable ahora esta codificada

# Entrenamiento del modelo

In [13]:
# Seleccionaremos el LGBMClassifier
# Inicializar el modelo LGBMClassifier
model = lgb.LGBMClassifier(max_depth=2, learning_rate=0.1, n_estimators=100)

# Entrenar el modelo
model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 169, number of negative: 286
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000499 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4548
[LightGBM] [Info] Number of data points in the train set: 455, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.371429 -> initscore=-0.526093
[LightGBM] [Info] Start training from score -0.526093
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

LGBMClassifier(max_depth=2)

In [26]:
# modelo SVM con kernel lineal
svm_nlinear = svm.SVC(kernel='rbf')
svm_nlinear.fit(X_train, y_train)

SVC()

In [17]:
# entrenando el modelo de arbol de decisión
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [18]:
# modelo SVM con kernel lineal
svm_linear = svm.SVC(kernel='linear')
svm_linear.fit(X_train, y_train)

SVC(kernel='linear')

# Evaluacion del modelo
* Primero evaluaremos los modelo con los datos de validacion y guardaremos los resultados
* Visalizaremos la matriz de confusion y el reporte de clasificacion del modelo para saber que clase clasifica mejor
* Finalmente, para el accuracy usaremos validacion cruzada para obtener una aproximacion al accuracy real del modelo

*hipotesis: se clasificara mejor a los tumores benignos*

## Modelo arbol de decision

In [24]:
# Predecir en el conjunto de prueba
y_pred = decision_tree.predict(X_test)
# Convertir las etiquetas a enteros
y_pred = y_pred.astype(int)
# Informes de clasificación y matrices de confusión
print("modelo")
print(classification_report(y_test, y_pred))
print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))

modelo
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        71
           1       0.93      0.93      0.93        43

    accuracy                           0.95       114
   macro avg       0.94      0.94      0.94       114
weighted avg       0.95      0.95      0.95       114

Matriz de Confusión:
 [[68  3]
 [ 3 40]]


In [36]:
# Predecir en el conjunto de prueba
y_pred = decision_tree.predict(X_test)

# Convertir las etiquetas a enteros
y_test = y_test.astype(int)
y_pred = y_pred.astype(int)
cv_scores = cross_val_score(decision_tree, X, y, cv=10)
# Evaluar la exactitud del modelo
accuracy = cv_scores.mean()

# Calcular recall y f1 especificando etiquetas numéricas
recall = recall_score(y_test, y_pred, labels=[0, 1], average='binary', zero_division=1)
f1 = f1_score(y_test, y_pred, labels=[0, 1], average='binary', zero_division=1)

# Imprimir la exactitud
print(f'Exactitud del modelo Arbol de decision: {accuracy * 100:.2f}%')

# Crear un DataFrame con las métricas de rendimiento
data = {
    'Metricas': ['Exactitud', 'Recall', 'F1 Score'],
    'Valores': [accuracy, recall, f1]
}
df = pd.DataFrame(data)

# Guardar los parámetros en un archivo CSV
df.to_csv('archivo.csv', index=False)

Exactitud del modelo Arbol de decision: 92.80%


## Modelo SVM lineal

In [25]:
# Predecir en el conjunto de prueba
y_pred = svm_linear.predict(X_test)
# Convertir las etiquetas a enteros
y_pred = y_pred.astype(int)
# Informes de clasificación y matrices de confusión
print("modelo")
print(classification_report(y_test, y_pred))
print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))

modelo
              precision    recall  f1-score   support

           0       0.95      0.99      0.97        71
           1       0.97      0.91      0.94        43

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114

Matriz de Confusión:
 [[70  1]
 [ 4 39]]


In [35]:
# Predecir en el conjunto de prueba
y_pred = svm_linear.predict(X_test)

# Convertir las etiquetas a enteros
y_test = y_test.astype(int)
y_pred = y_pred.astype(int)
cv_scores = cross_val_score(svm_linear, X, y, cv=10)
# Evaluar la exactitud del modelo
accuracy = cv_scores.mean()

# Calcular recall y f1 especificando etiquetas numéricas
recall = recall_score(y_test, y_pred, labels=[0, 1], average='binary', zero_division=1)
f1 = f1_score(y_test, y_pred, labels=[0, 1], average='binary', zero_division=1)

# Imprimir la exactitud
print(f'Exactitud del modelo SVM lineal: {accuracy * 100:.2f}%')

# Crear un DataFrame con las métricas de rendimiento
data = {
    'Metricas': ['Exactitud', 'Recall', 'F1 Score'],
    'Valores': [accuracy, recall, f1]
}
df = pd.DataFrame(data)

# Guardar los parámetros en un archivo CSV
df.to_csv('archivo.csv', index=False)

Exactitud del modelo SVM lineal: 95.43%


## Modelo SVM no lineal

In [28]:
# Predecir en el conjunto de prueba
y_pred = svm_nlinear.predict(X_test)
# Convertir las etiquetas a enteros
y_pred = y_pred.astype(int)
# Informes de clasificación y matrices de confusión
print("modelo")
print(classification_report(y_test, y_pred))
print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))

modelo
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        71
           1       1.00      0.86      0.93        43

    accuracy                           0.95       114
   macro avg       0.96      0.93      0.94       114
weighted avg       0.95      0.95      0.95       114

Matriz de Confusión:
 [[71  0]
 [ 6 37]]


In [34]:
# Predecir en el conjunto de prueba
y_pred = svm_nlinear.predict(X_test)

# Convertir las etiquetas a enteros
y_test = y_test.astype(int)
y_pred = y_pred.astype(int)
cv_scores = cross_val_score(svm_nlinear, X, y, cv=10)
# Evaluar la exactitud del modelo
accuracy = cv_scores.mean()

# Calcular recall y f1 especificando etiquetas numéricas
recall = recall_score(y_test, y_pred, labels=[0, 1], average='binary', zero_division=1)
f1 = f1_score(y_test, y_pred, labels=[0, 1], average='binary', zero_division=1)

# Imprimir la exactitud
print(f'Exactitud del modelo SVM no lineal: {accuracy * 100:.2f}%')

# Crear un DataFrame con las métricas de rendimiento
data = {
    'Metricas': ['Exactitud', 'Recall', 'F1 Score'],
    'Valores': [accuracy, recall, f1]
}
df = pd.DataFrame(data)

# Guardar los parámetros en un archivo CSV
df.to_csv('archivo.csv', index=False)

Exactitud del modelo SVM no lineal: 91.39%


## Modelo LGBM

In [20]:
# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)

# Convertir las etiquetas a enteros
y_test = y_test.astype(int)
y_pred = y_pred.astype(int)
# Informes de clasificación y matrices de confusión
print("modelo")
print(classification_report(y_test, y_pred))
print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))

modelo
              precision    recall  f1-score   support

           0       0.96      0.99      0.97        71
           1       0.98      0.93      0.95        43

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114

Matriz de Confusión:
 [[70  1]
 [ 3 40]]


In [38]:
# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)

# Convertir las etiquetas a enteros
y_test = y_test.astype(int)
y_pred = y_pred.astype(int)
cv_scores = cross_val_score(model, X, y, cv=10)
# Evaluar la exactitud del modelo
accuracy = cv_scores.mean()

# Calcular recall y f1 especificando etiquetas numéricas
recall = recall_score(y_test, y_pred, labels=[0, 1], average='binary', zero_division=1)
f1 = f1_score(y_test, y_pred, labels=[0, 1], average='binary', zero_division=1)

# Imprimir la exactitud
print(f'Exactitud del modelo LGBM: {accuracy * 100:.2f}%')

# Crear un DataFrame con las métricas de rendimiento
data = {
    'Metricas': ['Exactitud', 'Recall', 'F1 Score'],
    'Valores': [accuracy, recall, f1]
}
df = pd.DataFrame(data)

# Guardar los parámetros en un archivo CSV
df.to_csv('archivo.csv', index=False)

[LightGBM] [Info] Number of positive: 190, number of negative: 322
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000165 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5106
[LightGBM] [Info] Number of data points in the train set: 512, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.371094 -> initscore=-0.527527
[LightGBM] [Info] Start training from score -0.527527
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

## Resultado
La precision del mejor modelo segun validacion cruzada es LGBM con una exactitud del 96.48%

# Conclusiones
* El modelo LGBM tiene una exactitud bastante alta, se obtuvo un 96% a partir de la validacion cruzada la cual es una buena aproximacion a la exactitud real
* A partir de la evalucion de los modelos SVM lineal y no lineal concluimos que el problema es uno de clasificacion lineal, ya que se obtuvieron mejores resultados con un kernel lineal
* Sin embargo, como se hipotetisó el modelo se equivoco mas en los casos negativos, se tuvo un recall de 93% para esta clase. Pero, cuando detectó un tumor maligno su tuvo una exactitud mayor que aquella en el benigno, es decir si un tumor se detecta como maligno es extremadamente probable que sea así.
* Estos resultados mejoraran si se tienen mas datos, pero de momento se recomienda utilizar el modelo como descarte de malignidad en tumores, es decir, si se detecta que el tumor es maligno con un 98% de probabibilidad esto es verdad.

# Despliegue

In [ ]:
#Se guardará el modelo con mejor exactitud (LGBM) para desplagarlo en una pagina web
# Guardar el modelo entrenado
joblib.dump(model, 'model.pkl')
print("Modelo entrenado y guardado exitosamente.")

Modelo entrenado y guardado exitosamente.


El despliegue sera mediante una pagina web en donde se cargará un archivo csv con todas las caracteristicas de los tumores de pacientes y se daran las cantidades de tumores malignos y benignos si se analizar varios pacientes, pero tambien se acepta 1 solo paciente y se dará el resultado. Ademas se mostraran los parametros de accuracy recal y f1-score del modelo usado